In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import string

#nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import tokenize
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer, SentimentIntensityAnalyzer
from nltk.sentiment.util import *
import re

#sci-kit learn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from sklearn.metrics import (accuracy_score, roc_auc_score, confusion_matrix, roc_curve, auc, 
                             mean_squared_error, log_loss, precision_recall_curve, classification_report, 
                             precision_recall_fscore_support)
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, validation_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB



In [ ]:
# Input Data 
data_train = pd.read_csv("train.csv",encoding='latin1')
data_train.tail(10)

,meal_id,review,rating
12587,5258,"masakannya enak banget dan porsinya banyak, ??...",5.0
12588,4092,salad yg tidak ada dressing. sambal matahnya j...,2.0
12589,3545,"ayam kuning dada mentok mantap, kangkung pleci...",5.0
12590,3722,Rasanya cukup enak dan porsinya pas untuk saya.,4.0
12591,4092,"Dressingnya kurang, sambal matahnya full rawit...",3.0
12592,4921,worth it banget... rasa enak banget. porsi ken...,5.0
12593,4113,"nasi nya kurang mateng, masih ada keras2 diten...",3.0
12594,4125,"Sate nya kecil, kuskus nya aga kurang",3.0
12595,4376,rasa cumi kurang suka,3.0
12596,4114,bosen\n,3.0


In [ ]:
# Input Data 
data_validation = pd.read_csv("validation.csv",encoding='latin1')
data_validation.tail(10)

,meal_id,review,rating
6583,5839,ok,3.0
6584,5588,dagingnya terlalu sedikit,3.0
6585,1804,sayur kurang matang,3.0
6586,4493,porsinya sangat kurang,3.0
6587,4214,"Udangnya kecil sekali, tidak sesuai gambar. Sa...",3.0
6588,1804,"Porsi makanannya pas, rasanya sesuai",4.0
6589,5579,Makanan tidak datang.,1.0
6590,5588,daging alot tp enak,4.0
6591,6397,lauknya enak.. hanya saja nasinya agak keras..,3.0
6592,4138,"ternyata kurang suka sm mie shirataki, pengiri...",4.0


In [ ]:
# Input Data 
data_test = pd.read_csv("test.csv",encoding='latin1')
data_test.tail(10)

,meal_id,review
7536,7482,makasih boss
7537,5217,Daging ayamnya keras sekali :(
7538,5217,ayamnya enak. nasinya keras tapi.
7539,8519,ikan nya bau amis banget
7540,4222,"Pengirimannya cepat jam 10, rasanya ok"
7541,2045,"Pengantarannya cepat jam 10, rasanya mantap. Tks"
7542,8374,rasa diperbaiki
7543,8292,ok kok
7544,8468,enak
7545,3701,"porsi banyak, enak"


In [ ]:
data_train.review=data_train.review.astype(str)

In [ ]:
data_validation.review=data_validation.review.astype(str)

In [ ]:
data_test.review=data_test.review.astype(str)

In [ ]:
data_train.review = data_train.review.apply(lambda x:x.lower())

In [ ]:
data_validation.review = data_validation.review.apply(lambda x:x.lower())

In [ ]:
data_test.review = data_test.review.apply(lambda x:x.lower())

In [ ]:
 data_train.review = data_train.review.str.replace("[^\w\s]", "") #punctuation removal

In [ ]:
data_validation.review = data_validation.review.str.replace("[^\w\s]", "") #punctuation removal

In [ ]:
data_test.review = data_test.review.str.replace("[^\w\s]", "") #punctuation removal

In [ ]:
data_test = data_test[data_test['review'].notnull()]

In [ ]:
with open('stopword.txt','r') as file:
    stopwords = file.read().split('\n')
    
stopwords

['able',
 'about',
 'above',
 'according',
 'accordingly',
 'across',
 'actually',
 'after',
 'afterwards',
 'again',
 'against',
 "ain't",
 'all',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'an',
 'and',
 'another',
 'any',
 'anybody',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anyways',
 'anywhere',
 'apart',
 'appear',
 'appreciate',
 'appropriate',
 'are',
 "aren't",
 'around',
 'as',
 "a's",
 'aside',
 'ask',
 'asking',
 'associated',
 'at',
 'available',
 'awfully',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'believe',
 'below',
 'beside',
 'besides',
 'best',
 'better',
 'between',
 'beyond',
 'both',
 'brief',
 'but',
 'by',
 'came',
 'can',
 'cannot',
 'cant',
 "can't",
 'cause',
 'causes',
 'certain',
 'certainly',
 'changes',
 'clearly',
 "c'mon",
 'co',
 'com',
 'come',
 'comes',
 'concerning',
 'consequentl

In [ ]:
data_train.review = data_train.review.apply(lambda x: " ".join(x for x in x.split() if x not in stopwords))

In [ ]:
data_validation.review = data_validation.review.apply(lambda x: " ".join(x for x in x.split() if x not in stopwords))

In [ ]:
data_test.review = data_test.review.apply(lambda x: " ".join(x for x in x.split() if x not in stopwords))

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
data_train['review_tokenized'] = data_train.review.apply(lambda x: word_tokenize(x))

In [ ]:
data_validation['review_tokenized'] = data_validation.review.apply(lambda x: word_tokenize(x))

In [ ]:
data_test['review_tokenized'] = data_test.review.apply(lambda x: word_tokenize(x))

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
data_train['review_lemmatized'] = data_train.review_tokenized.apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [ ]:
data_validation['review_lemmatized'] = data_validation.review_tokenized.apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [ ]:
data_test['review_lemmatized'] = data_test.review_tokenized.apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [ ]:
data_train['review_lemmatized_train'] = data_train.review_lemmatized.apply(lambda x: ' '.join(x))

In [ ]:
data_validation['review_lemmatized_train'] = data_validation.review_lemmatized.apply(lambda x: ' '.join(x))

In [ ]:
data_test['review_lemmatized_train'] = data_test.review_lemmatized.apply(lambda x: ' '.join(x))

In [ ]:
x = (data_train['review_lemmatized_train'])
y = (data_train['rating'])
x_val = (data_validation['review_lemmatized_train'])
y_val = (data_validation['rating'])
x_test = (data_test['review_lemmatized_train'])

In [ ]:
x

0        ga ada rasa spesial sayur lodeh isinya cuma te...
1                                               ganti menu
2        kurang sreg sama gorengannya aja sih tempura k...
3        ayam dada nya agak kera dan nasinya cenderung ...
4                              pastanya enak bangettop deh
                               ...                        
12592          worth banget rasa enak banget porsi kenyang
12593    nasi nya kurang mateng masih ada keras2 diteng...
12594                 sate nya kecil kuskus nya aga kurang
12595                                rasa cumi kurang suka
12596                                                bosen
Name: review_lemmatized_train, Length: 12597, dtype: object

In [ ]:
y

0        3.0
1        3.0
2        3.0
3        4.0
4        5.0
        ... 
12592    5.0
12593    3.0
12594    3.0
12595    3.0
12596    3.0
Name: rating, Length: 12597, dtype: float64

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
x = cv.fit_transform(data_train['review_lemmatized_train']).toarray()
y = (data_train['rating'])
x_val = cv.fit_transform(data_validation['review_lemmatized_train']).toarray()
y_val = (data_validation['rating'])
x_testval = cv.fit_transform(data_test['review_lemmatized_train']).toarray()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(x, y) 

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
y_pred = classifier.predict(x_val)
y_pred

array([5., 3., 5., ..., 5., 3., 3.])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_val,y_pred))
print(classification_report(y_val,y_pred))
print(accuracy_score(y_val, y_pred))

[[  20    5   68   85   93]
 [  30    9   70   83  122]
 [ 122   23  348  466  641]
 [  83   21  620  749 1014]
 [  27    9  305  491 1089]]
              precision    recall  f1-score   support

         1.0       0.07      0.07      0.07       271
         2.0       0.13      0.03      0.05       314
         3.0       0.25      0.22      0.23      1600
         4.0       0.40      0.30      0.34      2487
         5.0       0.37      0.57      0.45      1921

    accuracy                           0.34      6593
   macro avg       0.24      0.24      0.23      6593
weighted avg       0.33      0.34      0.32      6593

0.33596238434703474


In [ ]:
y_pred2 = classifier.predict(x_testval)
y_pred2

array([5., 3., 5., ..., 5., 5., 5.])

In [ ]:
result_last = pd.DataFrame(data_test['review'])
last

,review,category
0,belum sampe,5.0
1,sampai pukul 1220 makanan belum sampai,3.0
2,enak sayang kulitnya keras bukan crunchy,5.0
3,udah 2 hari lupa dikasi sendok mulu nih,4.0
4,belum datang sampai sekarang,4.0
...,...,...
7541,pengantarannya cepat jam 10 rasanya mantap tks,3.0
7542,rasa diperbaiki,4.0
7543,kok,5.0
7544,enak,5.0


In [ ]:
result_last['category'] = pd.DataFrame(y_pred2)
result_last

,review,category
0,belum sampe,5.0
1,sampai pukul 1220 makanan belum sampai,3.0
2,enak sayang kulitnya keras bukan crunchy,5.0
3,udah 2 hari lupa dikasi sendok mulu nih,4.0
4,belum datang sampai sekarang,4.0
...,...,...
7541,pengantarannya cepat jam 10 rasanya mantap tks,3.0
7542,rasa diperbaiki,4.0
7543,kok,5.0
7544,enak,5.0


In [411]:
from google.colab import files
result_last.to_csv('prediction.csv') 
files.download('prediction.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>